# Example to run MCMC inversion
The inversion estimates the emissions deviation from some a priori value

In [ ]:
%load_ext autoreload
%autoreload 2

import pymc as pm
import matplotlib.pyplot as plt
import numpy as np
import aesara.tensor as at
from aesara import function
from pathlib import Path

from py12box_invert.paths import Paths
from py12box_invert.invert import Invert
from py12box_invert.plot import plot_mf, plot_emissions
from py12box import core

species = "CFC-11"
project_path = Paths.data / f"example/{species}"

In [ ]:
inv_mcmc_zero = Invert(project_path, species, method="mcmc_lat_gradient", n_threads=4, sensitivity_freq="yearly", start_year=2010, end_year=2020,
                sensitivity_from_zero=True)

In [ ]:
inv_mcmc_zero.create_matrices([70., 20., 10., 10.], from_zero=True) #These numbers don't mean anything

In [ ]:
inv_mcmc_zero.inversion()

# Reproducing analytical Bayesian version for testing

In [ ]:
inv_true = Invert(project_path, species, method="analytical_gaussian", n_threads=4, sensitivity_freq="yearly", start_year=2010, end_year=2020)
inv_true.run_inversion([70., 20., 10., 10.])

In [ ]:
inv = Invert(project_path, species, method="mcmc_analytical", n_threads=4, sensitivity_freq="monthly", start_year=2010, end_year=2020)
inv.run_inversion([70., 20., 10., 10.])